# Summary

`Try getting GPT Neo predictions using the Huggingface API.`

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import requests

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key
from jabberwocky.utils import load_huggingface_api_key
from htools import *

In [4]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [24]:
HF_API_KEY = load_huggingface_api_key()
HEADERS = {'Authorization': f'Bearer api_{HF_API_KEY}'}
URL_FMT = 'https://api-inference.huggingface.co/models/{}'
# These accept different parameters. For now just start with the basics, but
# keep these around in case I want to do something with them later.
_task2suff = {'generate': 'EleutherAI/gpt-neo-2.7B',
              'summarize': 'facebook/bart-large-cnn',
              'chat': 'microsoft/DialoGPT-large',
              'q&a': 'deepset/roberta-base-squad2'}
TASK2URL = DotDict({k: URL_FMT.format(v) for k, v in _task2suff.items()})
NEO_URL = URL_FMT.format(_task2suff['generate'])

In [54]:
def query_gpt_neo(prompt, top_k=None, top_p=None, temperature=1.0, 
                  repetition_penalty=None, max_tokens=256, api_key=None,
                  **kwargs):
    headers = {'Authorization': 
               f'Bearer api_{api_key or load_huggingface_api_key()}'}
    # Notice the names don't always align with parameter names - I wanted 
    # those to be more consistent with query_gpt3() function.
    data = {'inputs': prompt, 'top_k': top_k, 'top_p': top_p, 
            'temperature': temperature, 'max_new_tokens': max_tokens}
    r = requests.post(NEO_URL, headers=headers, data=json.dumps(data))
    return prompt, r.json()
#     return r.json()[0]['generated_text']

In [60]:
text = 'I love to'
prompt, res = query_gpt_neo(text, max_tokens=12)

print(res)

[{'generated_text': 'I love to think I can do things like this myself, but honestly, I know not if it\'s "fun" for me, my family, or my coworkers. I want to take it to the next level, but I\'m a bit nervous'}]


In [61]:
print(res[0]['generated_text'])

I love to think I can do things like this myself, but honestly, I know not if it's "fun" for me, my family, or my coworkers. I want to take it to the next level, but I'm a bit nervous


In [56]:
for row in res:
    print(row[0]['generated_text'])

KeyError: 0

In [33]:
dates_kwargs = load_prompt('short_dates', verbose=False)
res = query_gpt_neo(**dates_kwargs)

print(res)

Why does the world go so crazy so quickly???

If you are a person who has experienced any time of loss, pain or sadness you should know just how overwhelming life can be during that time.

The best way to get through an


In [37]:
res = query_gpt_neo('What makes fish so')

print(res)

Why does the world go so crazy so quickly???

If you are a person who has experienced any time of loss, pain or sadness you should know just how overwhelming life can be during that time.

The best way to get through an
